In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


from tqdm import tqdm

In [2]:
train = pd.read_csv("../input/train.csv", index_col=0)
test = pd.read_csv("../input/test.csv", index_col=0)


In [26]:
submission = pd.read_csv("../input/sample_submission.csv", index_col=0)

In [3]:
category = "wheezy-copper-turtle-magic"

In [22]:
# Config
features = [column for column in train.columns if column not in ["id", "target", category]]
probabilities = pd.Series(np.zeros(len(train)), index=train.index)
predictions = pd.Series(np.zeros(len(test)), index=test.index)

# Loop through wheezy-copper-turtle-magic
for i in tqdm(range(512)):
    # Subset train and test
    # where wheezy == i and features only
    train_ = train.loc[train[category] == i, :]
    test_ = test.loc[test[category] == i, :]
    
    # VarianceThreshold
    from sklearn.feature_selection import VarianceThreshold
    feature_selector = VarianceThreshold(threshold = 1.5).fit(train_.loc[:, features])
    train_2 = feature_selector.transform(train_.loc[:, features])
    test_2 = feature_selector.transform(test_.loc[:, features])
    
    # At this moment train_ and test_ contain all columns and only samples from wheezy == i
    # and train_2 and test_2 contain only selected columns and samples from wheezy == i    
    
    # Stratified k-fold
#     skf = sk.model_selection.StratifiedKFold(n_splits=10, random_state=26, shuffle=True)
    
#     for split_train_index, split_test_index in skf.split(train_2, train_["target"]):
        # QDA
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(train_2, train_["target"])

    # Getting probabilities of the test part of the split
    split_probabilities = qda.predict_proba(train_2)[:, 1]
    
    # Saving predictions
    probabilities[train_.index] += split_probabilities
    
    predictions[test_.index] += qda.predict_proba(test_2)[:, 1]

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:18<00:00, 24.63it/s]


In [10]:
print(sk.metrics.roc_auc_score(train["target"], probabilities))

0.9936729814493824


In [37]:
# Saving submission
all(submission.index == predictions.index)

predictions.name = "target"

df = pd.DataFrame(predictions)
print(df.head())
# print(submission)
df.to_csv("sample_submission.csv")

                                        target
id                                            
1c13f2701648e0b0d46d8a2a5a131a53  9.999926e-01
ba88c155ba898fc8b5099893036ef205  9.964730e-01
7cbab5cea99169139e7e6d8ff74ebb77  4.133498e-08
ca820ad57809f62eb7b4d13f5d4371a0  5.413707e-02
7baaf361537fbd8a1aaa2c97a6d4ccc7  3.434001e-01
